# Pre-Configuration

This notebook uses the jupyter_contrib_nbextensions package  
Install using: **pip install jupyter_contrib_nbextensions**


Configurable nbextensions used:
* Collapsible Headings
* Nbextensions edit menu item
* Execute Time
* Export Embedded HTML
* jypter-js-widgets/extension
* contrib_nbextensions_help_item
* Hide input
* jupyter_dash/main
* jupyter_server_proxy/tree
* Nbextensions dashboard tab
* plotlywidget/extension
* Table of contents (2)
* Variable Inspector

# Kraken.py Process

## Packages

In [ ]:
import time
import os
import requests
import urllib.parse
import hashlib
import hmac
import base64
import pandas as pd
import json
from cryptography.fernet import Fernet

## Inputs

In [ ]:
accepted_currencies = ['XDG',
                       'DOGE',
                       'USD',
                       'XMR',
                       'BTC',
                       'XBT',
                       'ETH',
                       'ADA',
                       'DOT']

fiat_currencies = ['USD','GBP']

pairs = ["DOGE/USD","XMR/USD","ADA/USD","BTC/USD","DOT/USD","ETH/USD"]

## Static Variables

In [ ]:
os.environ['ENCRYPT'] = 'True'
api_sec = b''
api_key = b''

In [ ]:
remap_assets = {'XDG':'DOGE',
                'XBT':'BTC'}

## Definitions

### Helper Functions

In [ ]:
def split_pair(pair, accepted_currencies=accepted_currencies):
    """
    Take a pair and split individually based on the list of accepted currencies.
    If one asset is provided, that asset will return in a duplicated array 
    e.g.
    split_pair('VTC') return ['VTC','VTC']
    """
    default_pair = {'found':False,
             'currency_short':'',
             'currency_long':''}
    return_pair = [default_pair.copy(), default_pair.copy()]
    
    for curr in accepted_currencies:
        for curr2 in [curr,'X'+curr,'XX'+curr,'Z'+curr]:
            if (pair.startswith(curr2)) & (return_pair[0]['found'] == False):
                return_pair[0]['found'] = True
                return_pair[0]['currency_short']=curr
                return_pair[0]['currency_long']=curr2
            if (pair.endswith(curr2)) & (return_pair[1]['found'] == False):
                return_pair[1]['found'] = True
                return_pair[1]['currency_short']=curr
                return_pair[1]['currency_long']=curr2
                
    if not (return_pair[0]['found'] == return_pair[1]['found'] == True):
        if return_pair[0]['found'] == return_pair[1]['found']:
            print(f'neither pair from {pair} were supported.')
        else:
            for i in return_pair:
                if i['found']:
                    print(f"Currency: {pair.replace(i['currency_long'],'')} not supported, consider adding to supported list")
    
    return [return_pair[0]['currency_short'],return_pair[1]['currency_short']]

In [ ]:
def parse_pairs_from_series(df, series_name):
    """
    Adjust a dataframe - take the series and split into two columns
    Returns the adjusted dataframe and the new column names
    """
    pair_cols = ['pair_1','pair_2']
    
    pair_df = df.pair.apply(split_pair).apply(pd.Series)
    pair_df.columns = pair_cols
    df = df.merge(pair_df,left_index=True,right_index=True)
    
    return df, pair_cols

In [ ]:
def remap_and_dedupe_assets(ls, remap_assets=remap_assets):     
    """
    Rename any assets in the list according to the default 'remap_assets' dict in the config
    Returns a list of unique assets
    """ 
    i=0
    for asset in ls:
        if asset in remap_assets.keys():
            ls[i]=remap_assets[asset]
        i+=1
    return list(set(ls))

In [ ]:
def remap_series(df, series_name, remap_assets=remap_assets):
    """
    Rename any assets in the dataframe according to the default 'remap_assets' dict in the config
    Returns the adjusted dataframe
    """ 
    for asset in remap_assets:
        df[series_name] = df[series_name].str.replace(asset,remap_assets[asset])
    return df

In [ ]:
def aggregate_balances_per_day_trade(df, currencies, pair_cols):
    """
    Return a dataframe which has a column for each asset in the currencies ls
    these columns will have the balance change for each date
    Works off the assumption that there are two columns (pair_cols)
        The first pair column is the valule for buy and sell volume
        The second pair column is the valule for fees and cost
        (THIS IS HOW IT'S SET TO BE IN KRAKEN)
    """
    
    df['vol'] = df['vol'].astype('float')
    df['fee'] = df['fee'].astype('float')
    df['cost'] = df['cost'].astype('float')
    for currency in currencies:
        df[currency]=0

        df.loc[(df.type=='sell') & (df[pair_cols[0]]==currency),
               currency] = df[currency] + (df.vol*-1)
        df.loc[(df.type=='buy') & (df[pair_cols[0]]==currency), 
               currency] = df[currency] + (df.vol)

        df.loc[df.pair_2==currency, currency] = df[currency] + (df.fee*-1)

        df.loc[(df.type=='sell') & (df[pair_cols[1]]==currency)
               , currency] = df[currency] + (df.cost)
        df.loc[(df.type=='buy') & (df[pair_cols[1]]==currency)
               , currency] = df[currency] + (df.cost*-1)
    
    return df.groupby('date').agg('sum')[currencies]

In [ ]:
def aggregate_balances_per_day_ledger(df, currencies, fee_series_name='fee', volume_series_name='amount'):
    """
    Return a dataframe which as a column for each asset in the currencies ls
    these columns will have the balance change for each date
    """
    
    df[fee_series_name] = df[fee_series_name].astype('float')
    for currency in currencies:
        if currency not in df.columns:
            df[currency]=0
        df.loc[df.asset==currency, currency] = df[currency] + (df[fee_series_name]*-1)
    
    df[volume_series_name] = df[volume_series_name].astype('float')
    for currency in list(set(fiat_currencies) & set(currencies)):
        if currency not in df.columns:
            df[currency]=0
            
        df.loc[(df.asset==currency) & (df.type=='deposit'),
               currency] = df[currency] + (df[volume_series_name])
        df.loc[(df.asset==currency) & (df.type=='withdrawal'), 
               currency] = df[currency] + (df[volume_series_name]*-1)

    return df.groupby('date').agg('sum')[currencies]

In [ ]:
def parse_api_results(resp, result_type):
    df = pd.DataFrame()
    for result in resp.json()['result'][result_type]:
        temp_df = pd.DataFrame(resp.json()['result'][result_type][result], index=[result])
        temp_df['time'] = pd.to_datetime(temp_df['time'], unit='s')  # convert to a readable date
        temp_df['date'] = temp_df.time.dt.date.astype('datetime64')

        df = pd.concat([df,temp_df])
    return df

In [ ]:
def load_key():
    if os.getenv('KEY') == None:
        key = input("""
Please enter password:
""")
        os.environ['KEY'] = key

    return os.environ['KEY'].encode('utf-8')

In [ ]:
def decrypt(e_passw):
    key = load_key()
    cipher_suite = Fernet(key)
    return cipher_suite.decrypt(e_passw)

### Kraken Private API 

In [ ]:
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode()

In [ ]:
def kraken_request(uri_path, data, api_key, api_sec):
    api_url = "https://api.kraken.com"

    if os.environ['ENCRYPT']:
        api_key = decrypt(api_key)
        api_sec = decrypt(api_sec)

    headers = {}
    headers['API-Key'] = api_key
    # get_kraken_signature() as defined in the 'Authentication' section
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)             
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

### Kraken Public API

In [ ]:
def fetch_OHLC_data(symbol, timeframe):
    """
    This function will get Open/High/Low/Close, Volume and tradecount data for the pair passed
    symbol must be in format XXX/XXX ie. BTC/USD
    """
    pair_split = symbol.split('/') 
    symbol = pair_split[0] + pair_split[1]
    url = f'https://api.kraken.com/0/public/OHLC?pair={symbol}&interval={timeframe}'
    response = requests.get(url)
    data=pd.DataFrame()
    if response.status_code == 200: 
        j = json.loads(response.text)
        result = j['result']
        keys = []
        for item in result:
            keys.append(item)
        if keys[0] != 'last':
            data = pd.DataFrame(result[keys[0]],columns=['unix', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'tradecount'])
        else:
            data = pd.DataFrame(result[keys[1]],columns=['unix', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'tradecount'])

        data['date'] = pd.to_datetime(data['unix'], unit='s')
        data['volume_from'] = data['volume'].astype(float) * data['close'].astype(float)

        if data is None:
            print("Did not return any data from Kraken for this symbol")
    else:
        print("Did not receieve OK response from Kraken API")
    return data

In [ ]:
def fetch_SPREAD_data(symbol):
    """
    This function will return the nearest bid/ask and calculate the spread for the symbol passed
    symbol must be in format XXX/XXX ie. BTC/USD
    """
    pair_split = symbol.split('/') 
    symbol = pair_split[0] + pair_split[1]
    url = f'https://api.kraken.com/0/public/Spread?pair={symbol}'
    response = requests.get(url)
    data=pd.DataFrame()
    if response.status_code == 200: 
        j = json.loads(response.text)
        result = j['result']
        keys = []
        for item in result:
            keys.append(item)
        if keys[0] != 'last':
            data = pd.DataFrame(result[keys[0]], columns=['unix', 'bid', 'ask'])
        else:
            data = pd.DataFrame(result[keys[1]], columns=['unix', 'bid', 'ask'])

        data['date'] = pd.to_datetime(data['unix'], unit='s')
        data['spread'] = data['ask'].astype(float) - data['bid'].astype(float)

        if data is None:
            print("Did not return any data from Kraken for this symbol")
    else:
        print("Did not receieve OK response from Kraken API")
    return data

## Process

### Trades Data

In [ ]:
# Pull trades
resp = kraken_request('/0/private/TradesHistory', 
                      {"nonce": str(int(1000*time.time())),
                       "trades": True},
                      api_key,
                      api_sec)
# Process trades
trades_df = parse_api_results(resp, 'trades')

In [ ]:
# Split pairs into individual columns
trades_df_pairs, pair_cols = parse_pairs_from_series(trades_df.copy(),'pair')

# Remap the old asset names
# Find all currencies listed in the pairs
currencies = []
for pair in pair_cols:
    trades_df_pairs = remap_series(trades_df_pairs, pair)
    currencies += trades_df_pairs[pair].drop_duplicates().tolist()
    
# dedupe them and remap if necessary
currencies = remap_and_dedupe_assets(currencies)

trades_df_bare = aggregate_balances_per_day_trade(trades_df_pairs, currencies, pair_cols)

### Ledger Data

In [ ]:
# Pull Ledger (includes withdrawals and deposits)
# We need this to include any fees, even when we were just moving shit about
ledger_df = pd.DataFrame()
for asset in currencies:
    resp = kraken_request('/0/private/Ledgers',
                          {"nonce": str(int(1000*time.time())),
                           "asset": asset,
                           "start": 1610124514},
                          api_key, 
                          api_sec)
    # Process ledger
    temp_df = parse_api_results(resp, 'ledger')
    temp_df = temp_df[temp_df.type != 'trade']    
    ledger_df = pd.concat([ledger_df,temp_df])

In [ ]:
ledger_df_asset = ledger_df.copy()
ledger_df_asset['asset'] = ledger_df_asset.asset.apply(split_pair).apply(pd.Series)[0]
ledger_df_asset = remap_series(ledger_df_asset, 'asset')
ledger_df_bare = aggregate_balances_per_day_ledger(ledger_df_asset, currencies, 'fee', 'amount')

### Combined Ledger & Trades

In [ ]:
# combine ledger and trades
balance_df = pd.concat([ledger_df_bare,trades_df_bare], axis=0)
balance_df = balance_df.groupby('date').agg(sum)
balance_df

### Daily Prices

In [ ]:
dta = {}

In [ ]:
# PULL DAILY PRICES FOR SPECIFIC PAIRS
# RETURNS A DICTIONARY WITH KEY: PAIR AND VALUE = DATAFRAME OF THE DAILY DATA

for pair in pairs:
    if pair not in dta.keys():
        print(f'new pair found! Pulling data for {pair}')
    dta[pair] = fetch_OHLC_data(symbol=pair, timeframe='1440')  # fetches daily data

In [ ]:
# Concat all into one dataframe
daily_values_df = pd.DataFrame()
for pair in pairs:
    tmpdf = dta[pair].copy()[['date','high','low']]  
    tmpdf['high'] = tmpdf['high'].astype(float)
    tmpdf['low'] = tmpdf['low'].astype(float)
    tmpdf[pair] = (tmpdf['high']+tmpdf['low'])/2  
    
    tmpdf = tmpdf[['date',pair]].set_index('date')
    daily_values_df = pd.concat([daily_values_df,tmpdf], axis = 1, sort=True, join='outer')

In [ ]:
# Pull Balances
resp = kraken_request('/0/private/Balance',
                      {"nonce": str(int(1000*time.time()))},
                      api_key,
                      api_sec)

account_balances = {}
if resp.json()['result']:
    account_balances = resp.json()['result']

In [ ]:
# RETURN:
# balance_df
# account_balances
# daily_values_df

# Charting

In [ ]:
# COMBINE DAILY SALES WITH OUR BALANCE POSITIONS
daily_data = daily_values_df.merge(balance_df, how='outer', left_index=True, right_index=True)

daily_data = daily_data.fillna(0)
for currency in currencies:
    daily_data[f'cum_{currency}'] = (daily_data[currency]).cumsum()
    daily_data = daily_data.drop(columns={currency}).rename(columns={f'cum_{currency}':currency})
    
    if currency not in fiat_currencies:
        daily_data[f'{currency}$'] = daily_data[currency]*daily_data[f'{currency}/USD']

In [ ]:
daily_data.tail()

In [ ]:
import plotly.graph_objects as go

df = daily_data[daily_data.index >= '2021-01-20'].copy()
    
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df.index, y=df["DOGE$"],
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(203,152,0,0.5)',
    fill='tonexty',
    name='DOGE',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["XMR$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,100,800,0.5)',
    name='XMR',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["ADA$"]+df["XMR$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,1000,800,0.5)',
    name='ADA',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["DOT$"]+df["ADA$"]+df["XMR$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,100,100,100.5)',
    name='DOT',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["ETH$"]+df["DOT$"]+df["XMR$"]+df["ADA$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(0,10,8000,0.5)',
    name='ETH',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["BTC$"]+df["ETH$"]+df["DOT$"]+df["XMR$"]+df["ADA$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,255,0)',
    fillcolor='rgba(2030,152,0,0.5)',
    name='BTC',
))
fig.add_trace(go.Scatter(
    x=df.index, y=df["USD"]+df["BTC$"]+df["ETH$"]+df["DOT$"]+df["XMR$"]+df["ADA$"]+df["DOGE$"],
    fill='tonexty',
    line_color='rgba(255,255,2550,0)',
    fillcolor='rgba(0,1000,80,0.5)',
    name='USD',
))

fig.update_traces(mode='lines')
fig.show()

# Appendix